### Imports

In [30]:
using Random
using Flux, Statistics, ProgressMeter, Plots, TaijaData, Distances, MLDatasets, CSV, DataFrames
using Flux: onehotbatch, onecold, crossentropy, logitcrossentropy, mse, throttle, update!, push!

include("attacks.jl")
include("train.jl")

adversarial_train (generic function with 1 method)

### Setting up MNIST data

In [31]:
train_df = CSV.read("../data/mnist_train.csv", DataFrame; header=false)
train_data = Matrix(train_df)

test_df = CSV.read("../data/mnist_test.csv", DataFrame; header=false)
test_data = Matrix(test_df)

x_train = train_data[:, 2:785]' ./ 255
y_train = train_data[:, 1]

x_test = test_data[:, 2:785]' ./ 255
y_test = test_data[:, 1]

x_train

784×60000 Matrix{Float64}:
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 ⋮                        ⋮              ⋱            ⋮                   
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0 

### Defining model architecture (MLP)

In [33]:
model_0 = Chain(
    Dense(784, 128, relu; init=Flux.glorot_normal),
    Dense(128,  10; init=Flux.glorot_normal)
)

model_1 = deepcopy(model_0)
model_2 = deepcopy(model_0)
model_3 = deepcopy(model_0)
model_4 = deepcopy(model_0)
model_5 = deepcopy(model_0)
model_6 = deepcopy(model_0)
model_7 = deepcopy(model_0)
model_8 = deepcopy(model_0)
model_9 = deepcopy(model_0)

spare_model = deepcopy(model_0)

Chain(
  Dense(784 => 128, relu),              # 100_480 parameters
  Dense(128 => 10),                     # 1_290 parameters
)                   # Total: 4 arrays, 101_770 parameters, 397.742 KiB.

### Training and attack hyperparameters

In [34]:
loss(x, y) = logitcrossentropy(x, y) # Not defining softmax in the model to help with CE
batch_size = 32
epochs = 20
clamp_range = (0, 1)
opt = Flux.Adam()

Adam(0.001, (0.9, 0.999), 1.0e-8, IdDict{Any, Any}())

In [36]:
model_10 = deepcopy(spare_model)

Chain(
  Dense(784 => 128, relu),              # 100_480 parameters
  Dense(128 => 10),                     # 1_290 parameters
)                   # Total: 4 arrays, 101_770 parameters, 397.742 KiB.

### Model Training

#### Classical training: No adversarial robustness in training loop

In [35]:
include("train.jl")

vanilla_losses = vanilla_train(model_0, loss, opt, x_train, y_train, epochs, batch_size, 0, 9)

Epoch: 1


┌ Warning: Layer with Float32 parameters got Float64 input.
│   The input will be converted, but any earlier layers may be very slow.
│   layer = Dense(784 => 128, relu)
│   summary(x) = 784×32 Matrix{Float64}
└ @ Flux /root/.julia/packages/Flux/97jpP/src/layers/stateless.jl:59


Average loss: 0.25276472611079615
Epoch: 2
Average loss: 0.11294162537232041


Progress:  10%|████▏                                    |  ETA: 0:03:15

Epoch: 3
Average loss: 0.07770973599161952


...

Epoch: 19
Average loss: 0.0065397715100571985


Progress:  95%|███████████████████████████████████████  |  ETA: 0:00:05

Epoch: 20
Average loss: 0.004725704526687029


Progress: 100%|█████████████████████████████████████████| Time: 0:01:40


20-element Vector{Any}:
 0.25276472611079615
 0.11294162537232041
 0.07770973599161952
 0.05718404680456345
 0.04443674259468292
 0.034626679830951614
 0.02747416676602637
 0.021954787962689685
 0.018791964783686367
 0.015182572519404736
 0.012922601495954828
 0.011461179091476757
 0.009432946283037017
 0.00934863440279005
 0.007818793054899166
 0.007534446189976491
 0.007224687753166594
 0.005619363352160386
 0.0065397715100571985
 0.004725704526687029

#### Adversarial training with PGD - varying strengths

In [37]:
adv_losses_1 = adversarial_train(model_1, loss, opt, x_train, y_train, epochs, batch_size, PGD, 0, 9, 0.03; attack_method=:PGD, iterations=3, step_size=0.01, clamp_range=clamp_range) # PGD with ϵ=0.03 and 3 steps
adv_losses_2 = adversarial_train(model_2, loss, opt, x_train, y_train, epochs, batch_size, PGD, 0, 9, 0.06; attack_method=:PGD, iterations=6, step_size=0.01, clamp_range=clamp_range) # PGD with ϵ=0.06 and 6 steps
adv_losses_3 = adversarial_train(model_3, loss, opt, x_train, y_train, epochs, batch_size, PGD, 0, 9, 0.09; attack_method=:PGD, iterations=9, step_size=0.01, clamp_range=clamp_range) # PGD with ϵ=0.09 and 9 steps
adv_losses_4 = adversarial_train(model_4, loss, opt, x_train, y_train, epochs, batch_size, PGD, 0, 9, 0.12; attack_method=:PGD, iterations=12, step_size=0.01, clamp_range=clamp_range) # PGD with ϵ=0.12 and 12 steps
adv_losses_5 = adversarial_train(model_5, loss, opt, x_train, y_train, epochs, batch_size, PGD, 0, 9, 0.15; attack_method=:PGD, iterations=15, step_size=0.01, clamp_range=clamp_range) # PGD with ϵ=0.15 and 15 steps
adv_losses_6 = adversarial_train(model_6, loss, opt, x_train, y_train, epochs, batch_size, PGD, 0, 9, 0.18; attack_method=:PGD, iterations=18, step_size=0.01, clamp_range=clamp_range) # PGD with ϵ=0.18 and 18 steps
adv_losses_7 = adversarial_train(model_7, loss, opt, x_train, y_train, epochs, batch_size, PGD, 0, 9, 0.21; attack_method=:PGD, iterations=21, step_size=0.01, clamp_range=clamp_range) # PGD with ϵ=0.21 and 21 steps
adv_losses_8 = adversarial_train(model_8, loss, opt, x_train, y_train, epochs, batch_size, PGD, 0, 9, 0.24; attack_method=:PGD, iterations=24, step_size=0.01, clamp_range=clamp_range) # PGD with ϵ=0.24 and 24 steps
adv_losses_9 = adversarial_train(model_9, loss, opt, x_train, y_train, epochs, batch_size, PGD, 0, 9, 0.27; attack_method=:PGD, iterations=27, step_size=0.01, clamp_range=clamp_range) # PGD with ϵ=0.27 and 27 steps
adv_losses_10 = adversarial_train(model_10, loss, opt, x_train, y_train, epochs, batch_size, PGD, 0, 9, 0.30; attack_method=:PGD, iterations=30, step_size=0.01, clamp_range=clamp_range) # PGD with ϵ=0.30 and 30 steps

Epoch: 1
Average loss: 0.503259606402119
Epoch: 2
Average loss: 0.22488409394274156


Progress:  10%|████▏                                    |  ETA: 0:01:56

Epoch: 3
Average loss: 0.15498502482349674


Progress:  15%|██████▏                                  |  ETA: 0:01:41

Epoch: 4
Average loss: 0.1151528409966578


...

Epoch: 19
Average loss: 1.8649051575342814


Progress:  95%|███████████████████████████████████████  |  ETA: 0:01:01

Epoch: 20
Average loss: 1.8605000098546347


Progress: 100%|█████████████████████████████████████████| Time: 0:20:32


20-element Vector{Any}:
 2.8106636142730714
 2.364638841311137
 2.2108070575078327
 2.118106114959717
 2.062717596880595
 2.0260639910380047
 1.994544005648295
 1.965775868288676
 1.9464088548660279
 1.9301481174468995
 1.9149268756866455
 1.9028233184178671
 1.8910125812530518
 1.8858642739613851
 1.880671960067749
 1.8747880641301473
 1.8730530181884766
 1.8694972616831462
 1.8649051575342814
 1.8605000098546347

In [38]:
using ONNXNaiveNASflux

ONNXNaiveNASflux.save("../models_new/model_0.onnx", model_0)
ONNXNaiveNASflux.save("../models_new/model_1.onnx", model_1)
ONNXNaiveNASflux.save("../models_new/model_2.onnx", model_2)
ONNXNaiveNASflux.save("../models_new/model_3.onnx", model_3)
ONNXNaiveNASflux.save("../models_new/model_4.onnx", model_4)
ONNXNaiveNASflux.save("../models_new/model_5.onnx", model_5)
ONNXNaiveNASflux.save("../models_new/model_6.onnx", model_6)
ONNXNaiveNASflux.save("../models_new/model_7.onnx", model_7)
ONNXNaiveNASflux.save("../models_new/model_8.onnx", model_8)
ONNXNaiveNASflux.save("../models_new/model_9.onnx", model_9)
ONNXNaiveNASflux.save("../models_new/model_10.onnx", model_10)
ONNXNaiveNASflux.save("../models_new/spare_model.onnx", spare_model)

407491